In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import geopandas as gpd
import rtree
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely import wkt
import descartes

import fiona
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', None)

sns.set_palette(sns.diverging_palette(10, 220, sep=80, n=3, center="dark"))

sns.set_context('talk')

import warnings 
warnings.filterwarnings('ignore')

In [2]:
aff = pd.read_csv('../data/open_data/csv/Affordable_Housing.csv')
aff['geometry'] = aff.apply(lambda row: Point(row.X, row.Y), axis=1)
aff = gpd.GeoDataFrame(aff)

In [3]:
poi = pd.read_csv('../data/open_data/csv/Points_of_Interest.csv')
poi['geometry'] = poi.apply(lambda row: Point(row.X, row.Y), axis=1)
poi = gpd.GeoDataFrame(poi)

In [50]:
poi

X          Y  OBJECTID  ADDRALIAS_ID  ADDRESS_ID  STATUS  \
0    -76.992044  38.881092    114241          4752      294505  ACTIVE   
1    -77.034829  38.901584    114242          2798      279976  ACTIVE   
2    -76.967292  38.858655    114243          7682      278278  ACTIVE   
3    -77.025967  38.916536    114244          9823      308944  ACTIVE   
4    -77.042588  38.895703    114246         12776      311026  ACTIVE   
5    -77.032968  38.895985    114247         12782      308612  ACTIVE   
6    -77.019381  38.845094    114249          6446      297321  ACTIVE   
7    -77.019730  38.828185    114260         11156      304868  ACTIVE   
8    -77.013977  38.869977    114261         11176      309788  ACTIVE   
9    -77.041158  38.893952    114262          6765      297693  ACTIVE   
10   -77.015520  38.900754    114266          8331      279959  ACTIVE   
11   -77.010361  38.898484    114267          6938      300403  ACTIVE   
12   -77.020984  38.924537    114268           475      188381  ACTIVE   
13   -77.018155  38.846494    114269         13551      312125  ACTIVE   
14   -77.015344  38.845291    114270         13557      312131  ACTIVE   
15   -77.023036  38.903596    114271          2985      290098  ACTIVE   
16   -77.038211  38.899132    114272           259      218720  ACTIVE   
17   -77.038201  38.899271    114273           154      218727  ACTIVE   
18   -77.038216  38.899714    114274          2882      218744  ACTIVE   
19   -77.038216  38.899714    114275           161      218744  ACTIVE   
20   -77.036890  38.985290    114276          8650      256418  ACTIVE   
21   -77.004778  38.891753    114277          8194      149847  ACTIVE   
22   -77.004778  38.891753    114278          2915      149847  ACTIVE   
23   -77.022598  38.901484    114279          7667      218759  ACTIVE   
24   -77.022598  38.901484    114280          1128      218759  ACTIVE   
25   -77.022598  38.901484    114281           165      218759  ACTIVE   
26   -77.034336  38.898150    114282          6757      297687  ACTIVE   
27   -77.025509  38.887962    114283          3078      293262  ACTIVE   
28   -77.049322  38.910343    114284          7060      301069  ACTIVE   
29   -77.047992  38.913216    114285          5097      295135  ACTIVE   
...         ...        ...       ...           ...         ...     ...   
9236 -77.012949  38.864134    170883         17212      277778  ACTIVE   
9237 -77.002945  38.879846    170884         17219      308596  ACTIVE   
9238 -77.005064  38.880231    170885         17220      310937  ACTIVE   
9239 -76.991257  38.854052    170886         17221      296115  ACTIVE   
9240 -76.991594  38.854528    170887         17222      298538  ACTIVE   
9241 -76.991832  38.854899    170888         17223      296119  ACTIVE   
9242 -76.990765  38.854869    170889         17224      298531  ACTIVE   
9243 -76.991024  38.854940    170890         17225      296116  ACTIVE   
9244 -76.991284  38.854956    170891         17226      296117  ACTIVE   
9245 -76.991618  38.855031    170892         17227      296118  ACTIVE   
9246 -76.991769  38.855380    170893         17228      296113  ACTIVE   
9247 -76.991479  38.855368    170894         17229      296112  ACTIVE   
9248 -76.991181  38.855352    170895         17230      296111  ACTIVE   
9249 -76.990899  38.855312    170896         17231      296110  ACTIVE   
9250 -76.990621  38.855237    171201         17232      296109  ACTIVE   
9251 -76.990304  38.855141    171202         17233      296108  ACTIVE   
9252 -76.990030  38.855060    171203         17234      296107  ACTIVE   
9253 -76.985123  38.853882    171204         17235      296101  ACTIVE   
9254 -76.985372  38.853842    171205         17236      296102  ACTIVE   
9255 -76.985692  38.853803    171206         17237      296103  ACTIVE   
9256 -76.985974  38.853916    171207         17238      296104  ACTIVE   
9257 -76.985557  38.854103    171208         17239      296105  ACTIVE   
9258 -76.985803  3

In [4]:
Census = gpd.read_file('../data/open_data/gis/Census_Blocks__2010/Census_Blocks__2010.shp')

In [5]:
Census.columns

Index(['OBJECTID', 'BLKGRP', 'BLOCK', 'GEOID', 'GEOID10', 'ALAND10',
       'AWATER10', 'P0010001', 'P0010002', 'P0010003', 'P0010004', 'P0010005',
       'P0010006', 'P0010007', 'P0010008', 'OP000001', 'OP000002', 'OP000003',
       'OP000004', 'P0020002', 'P0020005', 'P0020006', 'P0020007', 'P0020008',
       'P0020009', 'P0020010', 'OP00005', 'OP00006', 'OP00007', 'OP00008',
       'P0030001', 'P0030003', 'P0030004', 'P0030005', 'P0030006', 'P0030007',
       'P0030008', 'OP00009', 'OP00010', 'OP00011', 'OP00012', 'P0040002',
       'P0040005', 'P0040006', 'P0040007', 'P0040008', 'P0040009', 'P0040010',
       'OP000013', 'OP000014', 'OP000015', 'OP000016', 'H0010001', 'H0010002',
       'H0010003', 'ACRES', 'Shape_Leng', 'Shape_Area', 'SQMILES', 'geometry'],
      dtype='object')

In [6]:
Census.columns = ['OBJECTID', 'BLKGRP', 'BLOCK', 'GEOID', 'GEOID10', 'ALAND10',
       'AWATER10', 'TOTPOP', 'P0010002', 'P0010003', 'P0010004', 'P0010005',
       'P0010006', 'P0010007', 'P0010008', 'OP000001', 'OP000002', 'OP000003',
       'OP000004', 'P0020002', 'P0020005', 'P0020006', 'P0020007', 'P0020008',
       'P0020009', 'P0020010', 'OP00005', 'OP00006', 'OP00007', 'OP00008',
       'P0030001', 'P0030003', 'P0030004', 'P0030005', 'P0030006', 'P0030007',
       'P0030008', 'OP00009', 'OP00010', 'OP00011', 'OP00012', 'P0040002',
       'P0040005', 'P0040006', 'P0040007', 'P0040008', 'P0040009', 'P0040010',
       'OP000013', 'OP000014', 'OP000015', 'OP000016','Total_Housing_Units', 
       'Occupied_Housing_Units', 'Vacant_Housing_Units', 'ACRES', 'Shape_Length', 
       'Shape_Area', 'SQMILES', 'geometry']

In [7]:
MiniCensus = Census[['OBJECTID', 'BLKGRP', 'BLOCK', 'GEOID', 'GEOID10', 'ALAND10',
       'AWATER10', 'TOTPOP', 'Total_Housing_Units', 'Occupied_Housing_Units', 'Vacant_Housing_Units', 
       'ACRES', 'Shape_Length', 'Shape_Area', 'SQMILES', 'geometry']]

In [8]:
MiniCensus.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6507 entries, 0 to 6506
Data columns (total 16 columns):
OBJECTID                  6507 non-null int64
BLKGRP                    6507 non-null object
BLOCK                     6507 non-null object
GEOID                     6507 non-null object
GEOID10                   6507 non-null object
ALAND10                   6507 non-null int64
AWATER10                  6507 non-null int64
TOTPOP                    6507 non-null int64
Total_Housing_Units       6507 non-null int64
Occupied_Housing_Units    6507 non-null int64
Vacant_Housing_Units      6507 non-null int64
ACRES                     6507 non-null float64
Shape_Length              6507 non-null float64
Shape_Area                6507 non-null float64
SQMILES                   6507 non-null float64
geometry                  6507 non-null geometry
dtypes: float64(4), geometry(1), int64(7), object(4)
memory usage: 813.5+ KB


In [ ]:
MiniCensus.shape[0]

In [55]:
print(f"""Out of {MiniCensus.shape[0]} blocks, \
{MiniCensus.loc[MiniCensus['Occupied_Housing_Units'] > MiniCensus['Total_Housing_Units']].shape[0]} blocks \
have Occupied Housing Units > Total Housing Units and \
{MiniCensus.loc[MiniCensus['Total_Housing_Units'] != (MiniCensus['Occupied_Housing_Units']+MiniCensus['Vacant_Housing_Units'])].shape[0]} \
blocks have Total Housing Units != to Occupied Housing Units + Vacant Units""")


Out of 6507 blocks, 213 blocks have Occupied Housing Units > Total Housing Units and 547 blocks have Total Housing Units != to Occupied Housing Units + Vacant Units


In [34]:
MiniCensus.loc[MiniCensus['Occupied_Housing_Units'] > MiniCensus['Total_Housing_Units']].shape

(213, 18)

In [35]:
MiniCensus.loc[MiniCensus['Total_Housing_Units'] != (MiniCensus['Occupied_Housing_Units']+MiniCensus['Vacant_Housing_Units'])].shape
                                                     

(547, 18)

In [57]:
MiniCensus['PERC_OF_TOT_OCC'] = MiniCensus['Occupied_Housing_Units']/MiniCensus['Total_Housing_Units']
MiniCensus['PERC_OF_TOT_OCC'] = MiniCensus['PERC_OF_TOT_OCC'].fillna(0)

In [58]:
MiniCensus['OCC_PER_CAP'] = MiniCensus['Occupied_Housing_Units']/MiniCensus['TOTPOP']
MiniCensus['OCC_PER_CAP'] = MiniCensus['OCC_PER_CAP'].fillna(0)

In [59]:
MiniCensus.PERC_OF_TOT_OCC = MiniCensus.PERC_OF_TOT_OCC.replace(np.inf,0)
MiniCensus['OCC_PER_CAP'] = MiniCensus['OCC_PER_CAP'].replace(np.inf,0)

In [14]:
MiniCensus.OCC_PER_CAP.describe()

count    6507.000000
mean        0.410224
std         4.601279
min         0.000000
25%         0.000000
50%         0.351351
75%         0.445995
max       321.000000
Name: OCC_PER_CAP, dtype: float64

In [33]:
MiniCensus.to_file('../data/viz/census0.shp')

In [60]:
master = gpd.sjoin(poi, MiniCensus)

In [17]:
master.columns

Index(['X', 'Y', 'OBJECTID_left', 'ADDRALIAS_ID', 'ADDRESS_ID', 'STATUS',
       'ALIASTYPE', 'ALIASNAME', 'ADDRNUM', 'ADDRNUMSUFFIX', 'STNAME',
       'FULLADDRESS', 'METADATA_ID', 'SITE_ADDRESS_PK', 'ROADWAYSEGID', 'SSL',
       'TYPE_', 'ENTRANCETYPE', 'STREET_TYPE', 'QUADRANT', 'CITY', 'STATE',
       'SQUARE', 'SUFFIX', 'LOT', 'NATIONALGRID', 'ZIPCODE4', 'XCOORD',
       'YCOORD', 'STATUS_ID', 'ASSESSMENT_NBHD', 'ASSESSMENT_SUBNBHD',
       'CFSA_NAME', 'HOTSPOT', 'CLUSTER_', 'POLDIST', 'ROC', 'PSA', 'SMD',
       'CENSUS_TRACT', 'VOTE_PRCNCT', 'WARD', 'ZIPCODE', 'ANC',
       'NEWCOMMSELECT06', 'NEWCOMMCANDIDATE', 'FOCUS_IMPROVEMENT_AREA',
       'SE_ANNO_CAD_DATA', 'WARD_2002', 'WARD_2012', 'ANC_2002', 'ANC_2012',
       'SMD_2002', 'SMD_2012', 'geometry', 'index_right', 'OBJECTID_right',
       'BLKGRP', 'BLOCK', 'GEOID', 'GEOID10', 'ALAND10', 'AWATER10', 'TOTPOP',
       'Total_Housing_Units', 'Occupied_Housing_Units', 'Vacant_Housing_Units',
       'ACRES', 'Shape_Length', 'S

In [18]:
cols = list(MiniCensus.columns)
cols.append('ADDRESS_ID')
cols[0] = 'OBJECTID_right'
cols

['OBJECTID_right',
 'BLKGRP',
 'BLOCK',
 'GEOID',
 'GEOID10',
 'ALAND10',
 'AWATER10',
 'TOTPOP',
 'Total_Housing_Units',
 'Occupied_Housing_Units',
 'Vacant_Housing_Units',
 'ACRES',
 'Shape_Length',
 'Shape_Area',
 'SQMILES',
 'geometry',
 'PERC_OF_TOT_OCC',
 'OCC_PER_CAP',
 'ADDRESS_ID']

In [19]:
cols2 = cols.copy()
cols2.pop()
cols2

['OBJECTID_right',
 'BLKGRP',
 'BLOCK',
 'GEOID',
 'GEOID10',
 'ALAND10',
 'AWATER10',
 'TOTPOP',
 'Total_Housing_Units',
 'Occupied_Housing_Units',
 'Vacant_Housing_Units',
 'ACRES',
 'Shape_Length',
 'Shape_Area',
 'SQMILES',
 'geometry',
 'PERC_OF_TOT_OCC',
 'OCC_PER_CAP']

In [20]:
count = master[['GEOID10','ADDRESS_ID']].groupby(by='GEOID10').count()

In [21]:
count_poi = MiniCensus.merge(count, on='GEOID10')

In [22]:
count_poi.columns = ['OBJECTID', 'BLKGRP', 'BLOCK', 'GEOID', 'GEOID10', 'ALAND10',
       'AWATER10', 'TOTPOP', 'Total_Housing_Units', 'Occupied_Housing_Units',
       'Vacant_Housing_Units', 'ACRES', 'Shape_Length', 'Shape_Area',
       'SQMILES', 'geometry', 'PERC_OF_TOT_OCC', 'OCC_PER_CAP', 'ft_count']

In [23]:
count_poi['Total_Housing_Units'] = count_poi['Occupied_Housing_Units']+count_poi['Vacant_Housing_Units']

In [24]:
count_poi['PERC_OF_TOT_OCC'] = count_poi['Occupied_Housing_Units']/count_poi['Total_Housing_Units']
count_poi['PERC_OF_TOT_OCC'] = count_poi['PERC_OF_TOT_OCC'].fillna(0)

In [25]:
count_poi['OCC_PER_CAP'] = count_poi['Occupied_Housing_Units']/count_poi['TOTPOP']
count_poi['OCC_PER_CAP'] = count_poi['OCC_PER_CAP'].fillna(0)

In [28]:
count_poi_occupied = count_poi.loc[(count_poi.PERC_OF_TOT_OCC > .05)]

In [30]:
count_poi_occupied = count_poi_occupied.loc[count_poi.ft_count < 100]

In [61]:
sns.lmplot(x='PERC_OF_TOT_OCC',y='ft_count', data=count_poi_occupied);

In [32]:
count_poi_occupied.to_file('../data/viz/census1.shp')